In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    master("local").\
    appName('linear-regression-implementation-on-real-data').\
    getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv(
    path='D:/learn-ab/learning-PySpark/sample-data/ecommerce-customers.csv',
    inferSchema=True,
    header=True
)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(
    inputCols=[
        'Avg Session Length',
        'Time on App',
        'Time on Website',
        'Length of Membership'
    ],
    outputCol='features'
)

In [9]:
output = assembler.transform(data)

In [10]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [12]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [13]:
output.head(1)[0].features

DenseVector([34.4973, 12.6557, 39.5777, 4.0826])

In [14]:
final_data = output.select('features', 'Yearly Amount Spent')

In [15]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [16]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean| 504.05831269383134|
| stddev|  80.26258294170957|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean| 488.94915844147295|
| stddev|  76.43278490893105|
|    min| 256.67058229005585|
|    max|  700.9170916173961|
+-------+-------------------+



In [19]:
?LinearRegression

Init signature:
LinearRegression(
    *,
    featuresCol: str = 'features',
    labelCol: str = 'label',
    predictionCol: str = 'prediction',
    maxIter: int = 100,
    regParam: float = 0.0,
    elasticNetParam: float = 0.0,
    tol: float = 1e-06,
    fitIntercept: bool = True,
    standardization: bool = True,
    solver: str = 'auto',
    weightCol: Optional[str] = None,
    aggregationDepth: int = 2,
    loss: str = 'squaredError',
    epsilon: float = 1.35,
    maxBlockSizeInMB: float = 0.0,
)
Docstring:     
Linear regression.

The learning objective is to minimize the specified loss function, with regularization.
This supports two kinds of loss:

* squaredError (a.k.a squared loss)
* huber (a hybrid of squared error for relatively small errors and absolute error for     relatively large ones, and we estimate the scale parameter from training data)

This supports multiple types of regularization:

* none (a.k.a. ordinary least squares)
* L2 (ridge regression)
* L1 (Lasso)
* L

In [20]:
lr = LinearRegression(
    featuresCol='features',
    labelCol='Yearly Amount Spent',
    predictionCol='prediction'
)

In [21]:
lr_model = lr.fit(train_data)

In [22]:
test_res = lr_model.evaluate(test_data)

In [23]:
test_res.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -22.767904255971075|
| -1.0779664875109347|
|   2.494296072480097|
|  3.6433698387446043|
|   -18.9543709218849|
| -15.590825862340807|
|   17.63005724195068|
|  -4.249079666707132|
| -1.2502368076906691|
|  -5.704147078177755|
|   -8.82205067800595|
|  10.379210911989844|
| -1.6269635624084344|
|  12.246727602165095|
|  -7.570792882476667|
|    8.19049177297586|
| -18.904152842616668|
|  -4.389042621563817|
|   6.001777183470722|
|-0.11201397112978384|
+--------------------+
only showing top 20 rows



In [24]:
test_res.rootMeanSquaredError

10.721115767997041

In [25]:
test_res.r2

0.9801986109107287

In [26]:
test_res.r2adj

0.9796775217241689

In [27]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [28]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[31.1239743499119...|
|[31.2606468698795...|
|[31.3091926408918...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.7216523605090...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9096268275227...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9453957483445...|
|[31.9549038566348...|
|[31.9563005605233...|
|[31.9673209478824...|
|[31.9764800614612...|
|[32.0047530203648...|
+--------------------+
only showing top 20 rows



In [29]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.1239743499119...|509.71495809573685|
|[31.2606468698795...| 422.4045977444623|
|[31.3091926408918...|430.22642176745353|
|[31.5316044825729...|432.87223589061796|
|[31.5702008293202...| 564.8998630632898|
|[31.5741380228732...| 560.0000980229277|
|[31.6005122003032...|461.54279424914625|
|[31.6253601348306...| 380.5859804236313|
|[31.7216523605090...| 349.0271634395633|
|[31.8745516945853...|397.98939132444525|
|[31.8854062999117...|398.92532365048146|
|[31.9096268275227...| 553.0668247612493|
|[31.9120759292006...|389.16167986811615|
|[31.9262720263601...| 379.9582058421613|
|[31.9453957483445...| 664.5907168201286|
|[31.9549038566348...| 431.8073881669511|
|[31.9563005605233...| 566.0300845898155|
|[31.9673209478824...| 450.1388838612161|
|[31.9764800614612...| 324.5926688506295|
|[32.0047530203648...| 463.8579950917592|
+--------------------+------------